In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5

In [3]:
cwd = os.getcwd()
datafiles = os.listdir(os.path.join(cwd, 'data'))

In [4]:
fullstats = [i for i in datafiles if i.endswith('total.csv')]

In [5]:
df = pd.read_csv(os.path.join(cwd, 'data', fullstats[0]))
df['line'] = fullstats[0][:4]
df['timepoint'] = fullstats[0][5:6]
df['mouse'] = fullstats[0][:12]
df['channel'] = fullstats[0][-13:-10]
df.drop(columns=['Min', 'Mean', 'Max', ' '], inplace=True)

In [6]:
for file in fullstats[1:]:
    newdf = pd.read_csv(os.path.join(cwd, 'data', file))
    newdf['line'] = file[:4]
    newdf['timepoint'] = file[5:6]
    newdf['mouse'] = file[:12]
    newdf['channel'] = file[-13:-10]
    newdf.drop(columns=['Min', 'Mean', 'Max', ' '], inplace=True)
    df = df.append(newdf)

In [10]:
df.to_csv('unfiltered_dataframe.csv')

In [11]:
good_size = df[(df.Area > 12) & (df.Area < 100)]

In [12]:
good_size.to_csv('filtered_dataframe.csv')

In [13]:
print(df.shape)
print(good_size.shape)

(472171, 5)
(361911, 5)


In [14]:
good_size.groupby(['line', 'timepoint', 'channel']).count().Area

line  timepoint  channel
mcat  3          mko          8559
                 tfp         16934
                 yfp          8715
      8          mko          7587
                 tfp         14786
                 yfp          1105
robo  3          mko          3949
                 tfp          7274
                 yfp          6371
      8          mko         10372
                 tfp        226360
                 yfp         49899
Name: Area, dtype: int64

In [16]:
size_file = [i for i in datafiles if i.endswith('size.csv')]
size = pd.read_csv('./data/' + size_file[0])
size['line'] = size_file[0][:4]
size['timepoint'] = size_file[0][5:6]
size['mouse'] = size_file[0][:12]

In [17]:
for file in size_file[1:]:
    newdf = pd.read_csv(os.path.join(cwd, 'data', file))
    newdf['line'] = file[:4]
    newdf['timepoint'] = file[5:6]
    newdf['mouse'] = file[:12]
#    newdf.drop(columns=['Slice', 'Count', 'Average Size', '\%Area', 'Mean'], inplace=True)
    size = size.append(newdf)

In [18]:
size.drop(columns=['Slice','Count', 'Average Size', '%Area', 'Mean'], inplace=True)
size.reset_index(inplace=True, drop=True)
size.set_index(keys='mouse', inplace=True)

In [19]:
size.to_csv('slice_areas.csv')

In [20]:
mousecount = good_size.groupby(['mouse', 'channel']).count().Area
mousecount = mousecount.unstack()
mousecount = mousecount.merge(size, on='mouse')
mousecount['mko_c'] = mousecount['mko']/mousecount['Total Area']
mousecount['yfp_c'] = mousecount['yfp']/mousecount['Total Area']
mousecount['tfp_c'] = mousecount['tfp']/mousecount['Total Area']
mousecount.to_csv('full_count.csv')
avgmouse = mousecount.groupby(['line', 'timepoint']).mean()
avgmouse.reset_index(inplace=True)
avgmouse.set_index(avgmouse.line.str.cat(avgmouse.timepoint.str), inplace=True, drop=True)
collist = avgmouse.columns.tolist()
collist = [collist[3], collist[4], collist[2], collist[-1], collist[-2], collist[-3]]
avgmouse = avgmouse[collist]

In [21]:
(avgmouse.iloc[1]/avgmouse.iloc[0]).plot(kind='bar')
plt.title('Ratio of 8 week to 3 day counts RMCAT')

Text(0.5, 1.0, 'Ratio of 8 week to 3 day counts RMCAT')

In [22]:
(avgmouse.iloc[3]/avgmouse.iloc[2]).plot(kind='bar')
plt.title('Ratio of 8 week to 3 day counts RRobo')

Text(0.5, 1.0, 'Ratio of 8 week to 3 day counts RRobo')

In [ ]:
avgmouse['total'] = avgmouse.tfp + avgmouse.mko + avgmouse.yfp
avgmouse['total_c'] = avgmouse.tfp_c + avgmouse.yfp_c + avgmouse.mko_c

In [ ]:
(avgmouse.tfp_c/avgmouse.total_c).plot(kind='bar')
plt.xticks(rotation=45)
plt.title('Ratio of TFP')

In [ ]:
(avgmouse.mko_c/avgmouse.total_c).plot(kind='bar')
plt.xticks(rotation=45)
plt.title('Ratio of MKO')

In [ ]:
(avgmouse.yfp_c/avgmouse.total_c).plot(kind='bar')
plt.xticks(rotation=45)
plt.title('Ratio of YFP')

In [ ]:
avgmouse.yfp_corrected/avgmouse.total_corrected

In [ ]:
avgmouse.mko_corrected/avgmouse.total_corrected